<a href="https://colab.research.google.com/github/sauravkb94/Dog-and-cat-classification/blob/main/resent_Cat%26_dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import os
import zipfile
import random
import shutil

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt


In [ ]:
!wget https://unimibox.unimi.it/index.php/s/eNGYGSYmqynNMqF/download

--2022-12-16 14:34:35--  https://unimibox.unimi.it/index.php/s/eNGYGSYmqynNMqF/download
Resolving unimibox.unimi.it (unimibox.unimi.it)... 159.149.53.190
Connecting to unimibox.unimi.it (unimibox.unimi.it)|159.149.53.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘download.1’

download.1              [               <=>  ]  20.69M  1.78MB/s               ^C


In [ ]:
!unzip /content/download
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

Streaming output truncated to the last 5000 lines.
 extracting: CatsDogs/Dogs/5499.jpg  
 extracting: CatsDogs/Dogs/55.jpg    
 extracting: CatsDogs/Dogs/550.jpg   
 extracting: CatsDogs/Dogs/5500.jpg  
 extracting: CatsDogs/Dogs/5501.jpg  
 extracting: CatsDogs/Dogs/5502.jpg  
 extracting: CatsDogs/Dogs/5503.jpg  
 extracting: CatsDogs/Dogs/5504.jpg  
 extracting: CatsDogs/Dogs/5505.jpg  
 extracting: CatsDogs/Dogs/5506.jpg  
 extracting: CatsDogs/Dogs/5507.jpg  
 extracting: CatsDogs/Dogs/5508.jpg  
 extracting: CatsDogs/Dogs/5509.jpg  
 extracting: CatsDogs/Dogs/551.jpg   
 extracting: CatsDogs/Dogs/5510.jpg  
 extracting: CatsDogs/Dogs/5511.jpg  
 extracting: CatsDogs/Dogs/5512.jpg  
 extracting: CatsDogs/Dogs/5513.jpg  
 extracting: CatsDogs/Dogs/5514.jpg  
 extracting: CatsDogs/Dogs/5515.jpg  
 extracting: CatsDogs/Dogs/5516.jpg  
 extracting: CatsDogs/Dogs/5517.jpg  
 extracting: CatsDogs/Dogs/5518.jpg  
 extracting: CatsDogs/Dogs/5519.jpg  
 extracting: CatsDogs/Dogs/552.jpg   

**Create File DataFrame**

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
Image_Width=128
Image_Height=128
Image_Size=(Image_Width,Image_Height)
Image_Channels=3

In [ ]:
source_path = '/content/CatsDogs'

source_path_dogs = os.path.join(source_path, 'Dogs')
source_path_cats = os.path.join(source_path, 'Cats')

# Deletes all non-image files (there are two .db files bundled into the dataset)
!find /content/CatsDogs/ -type f ! -name "*.jpg" -exec rm {} +



In [ ]:
# os.listdir returns a list containing all files under the given path
print(f"There are {len(os.listdir(source_path_dogs))} images of dogs.")
print(f"There are {len(os.listdir(source_path_cats))} images of cats.")

There are 12500 images of dogs.
There are 12500 images of cats.


# Define root directory

In [ ]:
root_dir = '/content/CatsDogs'

# Empty directory to prevent FileExistsError is the function is run several times
if os.path.exists(root_dir):
  shutil.rmtree(root_dir)

In [ ]:
def create_train_val_dirs(root_path):
  """
  Creates directories for the train and test sets
  """  
  os.makedirs(root_path, 493 )
  train_dir = os.path.join(root_path, 'training')
  os.makedirs(train_dir, 493 )
  validation_dir = os.path.join(root_path, 'validation')
  os.makedirs(validation_dir, 493)
  train_cats_dir = os.path.join(train_dir, 'cats')
  os.makedirs(train_cats_dir, 493)
  train_dogs_dir = os.path.join(train_dir, 'dogs')
  os.makedirs(train_dogs_dir, 493)
  validation_cats_dir = os.path.join(validation_dir, 'cats')
  os.makedirs(validation_cats_dir, 493)
  validation_dogs_dir = os.path.join(validation_dir, 'dogs')
  os.makedirs(validation_dogs_dir, 493)

In [ ]:
try:
  create_train_val_dirs(root_path=root_dir)
except FileExistsError:
  print("You should not be seeing this since the upper directory is removed beforehand")

In [ ]:
for rootdir, dirs, files in os.walk(root_dir):
  for subdir in dirs:
    print(os.path.join(rootdir, subdir))

/content/CatsDogs/training
/content/CatsDogs/validation
/content/CatsDogs/training/dogs
/content/CatsDogs/training/cats
/content/CatsDogs/validation/dogs
/content/CatsDogs/validation/cats


In [ ]:
def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):
  """
  Splits the data into train and test sets
  """
  length = len([name for name in os.listdir(SOURCE_DIR)])
  newlength = int(length*SPLIT_SIZE)
  count = 1
  randomshuffle = random.sample([name for name in os.listdir(SOURCE_DIR)], length)
  for file in randomshuffle:
    size = os.path.getsize((os.path.join(SOURCE_DIR, file)))
    if(size>0):
      if(count<=newlength):
        copyfile((os.path.join(SOURCE_DIR, file)), (os.path.join(TRAINING_DIR, file)))
      else:
        copyfile ((os.path.join(SOURCE_DIR, file)), (os.path.join(VALIDATION_DIR, file)))
    else:
      print(file+" is zero length, so ignoring.")
    count=count+1

In [ ]:
# Define paths
CAT_SOURCE_DIR = "/content/CatsDogs/Cats"
DOG_SOURCE_DIR = "/content/CatsDogs/Dogs"

TRAINING_DIR = "/content/CatsDogs/training"
VALIDATION_DIR = "/content/CatsDogs/validation"

TRAINING_CATS_DIR = os.path.join(TRAINING_DIR, "cats/")
VALIDATION_CATS_DIR = os.path.join(VALIDATION_DIR, "cats/")

TRAINING_DOGS_DIR = os.path.join(TRAINING_DIR, "dogs/")
VALIDATION_DOGS_DIR = os.path.join(VALIDATION_DIR, "dogs/")


In [ ]:
# Empty directories in case you run this cell multiple times
if len(os.listdir(TRAINING_CATS_DIR)) > 0:
  for file in os.scandir(TRAINING_CATS_DIR):
    os.remove(file.path)
if len(os.listdir(TRAINING_DOGS_DIR)) > 0:
  for file in os.scandir(TRAINING_DOGS_DIR):
    os.remove(file.path)
if len(os.listdir(VALIDATION_CATS_DIR)) > 0:
  for file in os.scandir(VALIDATION_CATS_DIR):
    os.remove(file.path)
if len(os.listdir(VALIDATION_DOGS_DIR)) > 0:
  for file in os.scandir(VALIDATION_DOGS_DIR):
    os.remove(file.path)

In [ ]:
# Define proportion of images used for training
split_size = .9

In [ ]:
# Run the function
split_data (CAT_SOURCE_DIR, TRAINING_CATS_DIR, VALIDATION_CATS_DIR, split_size)
split_data (DOG_SOURCE_DIR, TRAINING_DOGS_DIR, VALIDATION_DOGS_DIR, split_size) 

666.jpg is zero length, so ignoring.
11702.jpg is zero length, so ignoring.


In [ ]:
print(f"\n\nOriginal cat's directory has {len(os.listdir(CAT_SOURCE_DIR))} images")
print(f"Original dog's directory has {len(os.listdir(DOG_SOURCE_DIR))} images\n")



Original cat's directory has 12500 images
Original dog's directory has 12500 images



In [ ]:
# Training and validation splits
print(f"There are {len(os.listdir(TRAINING_CATS_DIR))} images of cats for training")
print(f"There are {len(os.listdir(TRAINING_DOGS_DIR))} images of dogs for training")
print(f"There are {len(os.listdir(VALIDATION_CATS_DIR))} images of cats for validation")
print(f"There are {len(os.listdir(VALIDATION_DOGS_DIR))} images of dogs for validation")

There are 11249 images of cats for training
There are 11249 images of dogs for training
There are 1250 images of cats for validation
There are 1250 images of dogs for validation


In [ ]:
filenames=os.listdir("/content/CatsDogs/training")
categories=[]
for f_name in filenames:
    category=f_name.split('.')[0]
    if category=='dog':
        categories.append(1)
    else:
        categories.append(0)
df=pd.DataFrame({
    'filename':filenames,
    'category':categories
})

In [ ]:
df = pd.DataFrame()
df['filenam'] = filenames
df['category'] = categories
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,filenam,category
0,dogs,0
1,cats,0


In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.00001)
callbacks = [earlystop,learning_rate_reduction]

In [ ]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
  """
  Creates the training and validation data generators
  """

  # Instantiate the ImageDataGenerator class
  train_datagen = ImageDataGenerator(rescale=1/255)

  # Pass in the appropiate arguments to the flow_from_directory method
  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=20,
                                                      class_mode='binary',
                                                      target_size=(150, 150))

  # Instantiate the ImageDataGenerator class
  validation_datagen = ImageDataGenerator(rescale=1/255)

  # Pass in the appropiate arguments to the flow_from_directory method
  validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=20,
                                                                class_mode='binary',
                                                                target_size=(150, 150))
  return train_generator, validation_generator

In [ ]:
train_generator, validation_generator = train_val_generators(TRAINING_DIR, VALIDATION_DIR)

Found 22498 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [ ]:
def create_model():
  # DEFINE A KERAS MODEL TO CLASSIFY CATS V DOGS

  model = tf.keras.models.Sequential([ 
      tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2), 
      tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(128, (3,3), activation='relu'), 
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dense(1, activation='sigmoid') 
  ])

  
  model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.001),
                loss='binary_crossentropy',metrics=['accuracy']) 
  return model

In [ ]:
# Get the untrained model
model = create_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 74, 74, 16)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 34, 34, 64)        18496     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 17, 17, 64)      

In [ ]:
# Train the model
history = model.fit(train_generator,
                    epochs=15,
                    verbose=1,
                    validation_data=validation_generator)

Epoch 1/15
1125/1125 [==============================] - 627s 556ms/step - loss: 0.5611 - accuracy: 0.6993 - val_loss: 0.5387 - val_accuracy: 0.7392
Epoch 2/15
 780/1125 [===================>..........] - ETA: 3:00 - loss: 0.4453 - accuracy: 0.7930

KeyboardInterrupt: ignored

In [ ]:
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
#-----------------------------------------------------------
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.show()
print("")

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.show()

In [ ]:
input_path = []
label = []

for class_name in os.listdir("CatsDogs"):
    for path in os.listdir("CatsDogs/"+class_name):
        if class_name == 'Cats':
            label.append(0)
        else:
            label.append(1)
        input_path.append(os.path.join("CatsDogs", class_name, path))
      
print(input_path[0], label[0])

In [ ]:
df = pd.DataFrame()
df['images'] = input_path
df['label'] = label
df = df.sample(frac=1).reset_index(drop=True)
df.head()

In [ ]:
df.tail()

In [ ]:
for i in df['images']:
    if '.jpg' not in i:
        print(i)

In [ ]:
import PIL
l = []
for image in df['images']:
    try:
        img = PIL.Image.open(image)
    except:
        l.append(image)
l

In [ ]:
# delete db files
df = df[df['images']!='CatsDogs/Dogs/11702.jpg']
df = df[df['images']!='CatsDogs/Cats/666.jpg']
len(df)

In [ ]:
import seaborn as sns
sns.countplot(df['label'])

In [ ]:
# input split
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
inputs = tf.keras.Input(shape=(128,128,3))
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# final model
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
#callbacks and learning rate
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.00001)
callbacks = [earlystop,learning_rate_reduction]

In [ ]:
import pandas as pd

In [ ]:
import cv2

In [ ]:
train_dir ="/content/CatsDogs" 
path = os.path.join(train_dir)

for p in os.listdir(path):
    category = p.split(".")[0]
    img_array = cv2.imread(os.path.join(path,p),cv2.IMREAD_GRAYSCALE)
    new_img_array = cv2.resize(img_array, dsize=(80, 80))
    plt.imshow(new_img_array,cmap="gray")
    break

In [ ]:
train_df,validate_df = train_test_split(df,test_size=0.20,
  random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

total_train=train_df.shape[0]
total_validate=validate_df.shape[0]
batch_size=15

In [ ]:
train_df

In [ ]:
train_data, validation_data = train_test_split(df, test_size=0.70, shuffle=True,random_state=1)

train_data = train_data.reset_index(drop=True)
validation_data = validation_data.reset_index(drop=True)

final_train_data = train_data.shape[0]
final_validation_data = validation_data.shape[0]
batch_size=15

In [ ]:
train_data

In [ ]:
validation_data

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2
)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_data,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

In [ ]:
val_images = train_generator.flow_from_dataframe(
    dataframe=train_data,
    x_col='Images',
    y_col='label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation')

In [ ]:
test_images = test_generator.flow_from_dataframe(
    dataframe=validation_data,
     x_col='Images',
    y_col='label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)

Training

In [ ]:
inputs = tf.keras.Input(shape=(128,128,3))
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# final model
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
history = model.fit (train_images,val_images, epochs = 30,
    validation_data= test_images,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            patience=3
        )
    ]
)

Define callbacks and learning rate

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.00001)
callbacks = [earlystop,learning_rate_reduction]

Manage data:

**Load Image Data**

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2
)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [ ]:
train_images = train_generator.flow_from_dataframe( dataframe= train_data,
                                                     target_size=(256, 256), 
                                                     color_mode='rgb',
                                                     class_mode='binary',
                                                     batch_size=36,
                                                     shuffle=True, subset='training')

In [ ]:
train_generator = train_datagen.flow_from_dataframe (train_data, directory ="/content/CatsDogs",
                                                  target_size= (256, 256),
                                                  color_mode='rgb',
                                                  class_mode='binary',
                                                  batch_size = 36, shuffle=True, subset='training' )


In [ ]:
val_data = train_generator.flow_from_directory (directory = "/content/CatsDogs",
                                                     target_size=(256, 256), 
                                                     color_mode='rgb',
                                                     class_mode='binary',
                                                     batch_size=36,
                                                     shuffle=True, subset='validation'                           
                                               )

In [ ]:
test_data

**Filtering the data**

In [ ]:
for i in df['Images']:
    if '.jpg' not in i:
        print(i)

In [ ]:
import PIL
l = []
for image in df['Images']:
    try:
        img = PIL.Image.open(image)
    except:
        l.append(image)
l

In [ ]:
# delete db files
df = df[df['Images']!='CatsDogs/Dogs/11702.jpg']
df = df[df['Images']!='CatsDogs/Cats/666.jpg']
len(df)

In [ ]:
import seaborn as sns
sns.countplot(df['label'])

**Visulizing the data**

In [ ]:
t_img , label = train_data.next()

In [ ]:
  t_img.shape

In [ ]:
def plotImages(img_arr, label):
  """
  input : image_array
  output: plot images
  """

  for idx, img in enumerate(img_arr) :

    if idx <= 10 :

      plt.figure(figsize = (5,5))
      plt.imshow(img)
      plt.title(img.shape)
      plt.axis = False
      plt.show()

In [ ]:
plotImages(t_img, label)

In [ ]:
#Normalize data
X = X/255.0

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

**Model Creation**

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

mc  = ModelCheckpoint(filepath = "./test_model.h5",
                      monitor="accuracy",
                      verbose=1,
                      save_best_only = True)

es = EarlyStopping(monitor = "accuracy", 
                   min_delta= 0.01, 
                   patience = 5, 
                   verbose = 1)                      
                              
callbacks = [mc, es]                             

In [ ]:
his = model.fit (train_data, val_data, verbose=1 , epochs = 30,callbacks = callbacks) 
                          

In [ ]:
epochs = 30
batch_size = 32
model.fit_generator (train_data, verbose=1 , 
          batch_size=batch_size , 
          steps_per_epoch=10,
          callbacks = callbacks ,
          validation_data=(train_data,val_data))        

In [ ]:
validation_steps=30, verbose=1, epochs= 50)  
callbacks = [checkpoint, reduce_lr]

In [ ]:
from keras.models import load_model

model = load_model("/content/test_model.h5")

In [ ]:
h = his.history
h.keys()

In [ ]:
plt.plot(h['loss'], c= "red")
plt.plot(h['accuracy'], c = "blue",)

plt.title("loss vs accuracy")
plt.show()

In [ ]:
print('Test score:', h['loss'])    #Loss on test
print('Test accuracy:', (h['accuracy']* 100))

**Validate our Model**

In [ ]:
# path for the image  to see is it predics correct class

path = "/content/CatsDogs/Cats/10009.jpg"
img = load_img(path, target_size=(256, 256) )

i = img_to_array(img)

i = preprocess_input(i)

input_arr = np.array([i])
input_arr.shape

pred = np.argmax(model.predict(input_arr))

print(pred)

if pred == 0:
  print("The image is of cat")
else:
  print("the image is of dog")

plt.imshow(input_arr[0])
plt.title("input image")
plt.axis = False
plt.show()



In [ ]:
test_filenames = os.listdir("/content/CatsDogs")
test_data = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_data.shape[0]

Creating Test data generator:

In [ ]:
generate_test_data= ImageDataGenerator(rescale=1./255)
test_gen = generate_test_data.flow_from_dataframe(
    test_data, 
    "/content/CatsDogs", 
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=(224, 224),
    batch_size=32,
    shuffle=False
)

In [ ]:
pred=model.predict_classes([im])[0]

In [ ]:
# path for the image  to see is it predics correct class

path= "/content/CatsDogs/Cats/10004.jpg"
img = load_img(path, target_size=(244,244) )

i = img_to_array(img)

input_arr = np.array([i])
input_arr.shape

pred = np.argmax(model.predict(input_arr))


if pred == 0:
  print("This image is of cat")
else:
  print ("This image is of dog")

#to display the imgae 
plt.imshow(input_arr[0])
plt.title("input image")
plt.show()

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Dense, Flatten, Dropout
from keras.optimizer_v1 import Adam
from keras.callbacks import TensorBoard

In [ ]:
#need to normalize 
def process(image, label):
  image = tf.cast(image/255. , tf.float32)
  return image , label

In [ ]:
train_generator = train_dataset.map(process)
train_data = validation_dataset.map(process)

In [ ]:
### Split the dataset into independent and dependent dataset
X=df['images'].tolist()
y=df['label'].tolist()

In [ ]:
### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

Model Creation

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import TensorBoard

In [ ]:
#@title
import tensorflow as tf
print(tf.__version__)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [ ]:
from tensorflow.keras import layers

In [ ]:
model=Sequential()

In [ ]:
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])


In [ ]:
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(Image_Width,Image_Height,Image_Channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

In [ ]:
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(Image_Width,Image_Height,Image_Channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

In [ ]:
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

In [ ]:
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
  optimizer='rmsprop',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

In [ ]:
model.summary()